In [6]:
# !pip install selenium
# !pip install webdriver-manager

In [7]:
import pandas as pd
import numpy as np
import gradio as gr
import yfinance as yf
import requests

from bs4 import BeautifulSoup
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from datetime import datetime, timedelta

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Selenium WebDriver 설정
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

# Yahoo Finance 나스닥 100 구성 페이지로 이동
url = 'https://finance.yahoo.com/quote/%5ENDX/components/'
driver.get(url)

# 페이지 로드 대기
time.sleep(5)  # 5초 대기, 페이지가 완전히 로드되도록 함

# 종목 티커가 들어있는 테이블을 찾기
tickers = []
rows = driver.find_elements(By.XPATH, '//table/tbody/tr')

# 상위 30개 종목의 티커를 추출
for row in rows[:5]:
    ticker = row.find_element(By.XPATH, './td[1]').text  # 첫 번째 열에서 티커 가져오기
    tickers.append(ticker)

# 결과 출력
print(tickers)

# 드라이버 종료
driver.quit()

KeyboardInterrupt: 

In [ ]:
Stacktrace:
#0 0x556bad2ab2da <unknown>
#1 0x556bacf79200 <unknown>
#2 0x556bacfc8aa2 <unknown>
#3 0x556bacfc8d41 <unknown>
#4 0x556bacfbd386 <unknown>
#5 0x556bacfed8fd <unknown>
#6 0x556bacfbd279 <unknown>
#7 0x556bacfeda9e <unknown>
#8 0x556bad00bdfd <unknown>
#9 0x556bacfed673 <unknown>
#10 0x556bacfbb473 <unknown>
#11 0x556bacfbc47e <unknown>
#12 0x556bad2720db <unknown>
#13 0x556bad276071 <unknown>
#14 0x556bad25e9d5 <unknown>
#15 0x556bad276bf2 <unknown>
#16 0x556bad243b6f <unknown>
#17 0x556bad29a248 <unknown>
#18 0x556bad29a417 <unknown>
#19 0x556bad2aa0cc <unknown>
#20 0x7f7797d5fac3 <unknown

In [25]:
!pip install prophet


In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import yfinance as yf
import gradio as gr
import matplotlib.pyplot as plt
from prophet import Prophet  # prophet을 사용합니다.
from datetime import datetime, timedelta
import pandas as pd

# Selenium으로 Yahoo Finance에서 상위 30개 종목 가져오기
def get_tickers():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = 'https://finance.yahoo.com/quote/%5ENDX/components/'
    driver.get(url)
    time.sleep(5)  # 페이지 로드 대기

    tickers = []
    rows = driver.find_elements(By.XPATH, '//table/tbody/tr')
    for row in rows[:30]:  # 상위 30개만 추출
        ticker = row.find_element(By.XPATH, './td[1]').text
        tickers.append(ticker)

    driver.quit()
    return tickers

# 스크래핑한 티커 사용
tickers = get_tickers()

# Yahoo Finance에서 주식 데이터 가져오기
def fetch_stock_data(ticker):
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    data = yf.download(ticker, start="2014-01-01", end=end_date)
    return data

# Prophet을 이용한 시계열 예측 수행
def prophet_time_series(data):
    df = data.reset_index()[['Date', 'Close']]
    df.columns = ['ds', 'y']  # Prophet이 요구하는 포맷 (ds: 날짜, y: 타겟값)
    
    # Prophet 모델 학습
    model = Prophet()
    model.fit(df)
    
    # 미래 30일 예측
    future_dates = model.make_future_dataframe(periods=30)
    forecast = model.predict(future_dates)

    # 미래 30일 예측 데이터만 필터링
    prediction_df = forecast[['ds', 'yhat']].loc[forecast['ds'] > df['ds'].max()]  # 학습 데이터의 마지막 날짜 이후 데이터만 선택
    return prediction_df, model


# 시각화 함수: 예측 결과
def plot_predictions(predictions):
    file_path = 'predictions_plot.png'
    plt.figure(figsize=(10, 6))
    plt.plot(predictions['ds'], predictions['yhat'], label='Predicted', linestyle='--', marker='x')
    plt.xlabel('Date')
    plt.ylabel('Predicted Close Price')
    plt.title('Predicted Close Prices for the Next 30 Days')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(file_path)
    plt.close()
    return file_path

# 분석 함수
def analyze_stock(selected_ticker):
    stock_data = fetch_stock_data(selected_ticker)
    
    # Prophet을 통해 종가 예측
    predictions, model = prophet_time_series(stock_data)

    # 예측 시각화
    prediction_plot = plot_predictions(predictions)

    return prediction_plot

# Gradio 인터페이스 구성
with gr.Blocks() as mini_pro:
    with gr.Row():
        with gr.Column():
            # 스크래핑한 상위 30개 종목을 선택할 수 있는 드롭다운
            stock_selector = gr.Dropdown(choices=tickers, label="주식 종목 선택")
            run_button = gr.Button("30일 예측 및 시각화")

    with gr.Row():
        # 예측 결과 시각화 이미지 출력
        with gr.Column():
            prediction_image_output = gr.Image(label="Predicted Close Prices (30일 예측)")

    # 버튼 클릭 시 분석 실행
    def run_analysis(selected_ticker):
        try:
            prediction_plot = analyze_stock(selected_ticker)
            return prediction_plot
        except Exception as e:
            return f"Error during analysis: {e}"

    # 버튼 클릭 시 실행 및 업데이트
    run_button.click(
        run_analysis,
        inputs=[stock_selector],
        outputs=[prediction_image_output]
    )

# Gradio 앱 실행
mini_pro.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


[*********************100%***********************]  1 of 1 completed
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjfy2awmh/f77maw13.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpjfy2awmh/8s744avt.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/home/user/miniforge3/envs/ag/lib/python3.10/site-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=69416', 'data', 'file=/tmp/tmpjfy2awmh/f77maw13.json', 'init=/tmp/tmpjfy2awmh/8s744avt.json', 'output', 'file=/tmp/tmpjfy2awmh/prophet_modeli0vycy6r/prophet_model-20241017172135.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:21:35 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:21:36 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


In [30]:
mini_pro.close()

Closing server running on port: 7862


# autogluon에서 time_series사용

In [31]:
import pandas as pd
import gradio as gr
import yfinance as yf
import matplotlib.pyplot as plt
from autogluon.timeseries import TimeSeriesPredictor, TimeSeriesDataFrame
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime, timedelta
import time

# Selenium으로 Yahoo Finance에서 상위 30개 종목 가져오기
def get_tickers():
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    url = 'https://finance.yahoo.com/quote/%5ENDX/components/'
    driver.get(url)
    time.sleep(5)  # 페이지 로드 대기

    tickers = []
    rows = driver.find_elements(By.XPATH, '//table/tbody/tr')
    for row in rows[:30]:  # 상위 30개만 추출
        ticker = row.find_element(By.XPATH, './td[1]').text
        tickers.append(ticker)

    driver.quit()
    return tickers

# 스크래핑한 티커 사용
tickers = get_tickers()

# Yahoo Finance에서 주식 데이터 가져오기
def fetch_stock_data(ticker):
    print(f"Fetching stock data for {ticker}...")
    end_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')
    start_date = "2014-01-01"
    data = yf.download(ticker, start=start_date, end=end_date)
    print(f"Downloaded {len(data)} rows of data for {ticker}.")
    return data

# AutoGluon을 이용한 시계열 예측 수행
def autogluon_time_series(data, ticker):
    # 데이터프레임에서 datetime 타입으로 변환
    data.index = pd.to_datetime(data.index)
    
    # 학습 데이터 구성 (TimeSeriesDataFrame 형식으로 변환)
    train_data = pd.DataFrame({
        "timestamp": data.index,
        "target": data['Close'],
        "item_id": ticker  # 종목 코드로 item_id 설정
    }).reset_index(drop=True)

    # TimeSeriesDataFrame으로 변환
    train_data_ts = TimeSeriesDataFrame(train_data)

    # AutoGluon TimeSeries 모델 생성 및 학습
    predictor = TimeSeriesPredictor(
        prediction_length=30,  # 30일 예측
        time_limit=300,  # 학습 제한 시간 설정
        freq="1D"  # 데이터가 일 단위로 수집됨을 지정
    ).fit(train_data_ts)
    
    # 예측할 데이터 생성 (미래 데이터 예측)
    future_forecast = predictor.predict(train_data_ts)
    
    return future_forecast

# 시각화 함수: 예측 결과
def plot_predictions(forecast, actual_data, ticker):
    file_path = f'predictions_plot_{ticker}.png'
    plt.figure(figsize=(10, 6))
    plt.plot(actual_data.index, actual_data['Close'], label='Actual', marker='o')
    plt.plot(forecast.index, forecast['mean'], label='Predicted', linestyle='--', marker='x')
    plt.xlabel('Date')
    plt.ylabel('Close Price')
    plt.title(f'Predicted Close Prices for {ticker} (Next 30 Days)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(file_path)
    plt.close()
    return file_path

# 분석 함수
def analyze_stock(selected_ticker):
    stock_data = fetch_stock_data(selected_ticker)
    
    # AutoGluon을 통해 종가 예측
    forecast = autogluon_time_series(stock_data, selected_ticker)

    # 예측 시각화
    prediction_plot = plot_predictions(forecast, stock_data, selected_ticker)

    return prediction_plot

# Gradio 인터페이스 구성
with gr.Blocks() as mini_pro:
    with gr.Row():
        with gr.Column():
            # 스크래핑한 상위 30개 종목을 선택할 수 있는 드롭다운
            stock_selector = gr.Dropdown(choices=tickers, label="주식 종목 선택")
            run_button = gr.Button("30일 예측 및 시각화")

    with gr.Row():
        # 예측 결과 시각화 이미지 출력
        with gr.Column():
            prediction_image_output = gr.Image(label="Predicted Close Prices (30일 예측)")

    # 버튼 클릭 시 분석 실행
    def run_analysis(selected_ticker):
        try:
            prediction_plot = analyze_stock(selected_ticker)
            return prediction_plot  # 이미지 파일 경로만 반환
        except Exception as e:
            print(f"Error during analysis: {e}")
            return None  # 예외 발생 시 이미지 대신 None 반환

    # 버튼 클릭 시 실행 및 업데이트
    run_button.click(
        run_analysis,
        inputs=[stock_selector],
        outputs=[prediction_image_output]
    )

# Gradio 앱 실행
mini_pro.launch()


* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [30]:
mini_pro.close()

Closing server running on port: 7862
